# Extract the rectangular network of edges and nodes based on its geographic location.

# osm-network-dl

## Code for downloading a network from open street maps using the osmnx library. Input is pair of latitudes and longitudes specifying the region of the network of interest.


## Currently, input is done during runtime via keyboard inputs.


# Install all the dependencies and libraries
pip install wheel
pip install pipwin
pipwin install numpy
pipwin install pandas
pipwin install shapely
pipwin install gdal
pipwin install fiona
pipwin install pyproj
pipwin install six
pipwin install rtree
pipwin install geopandas
pip install geopandas
pip install osmx

## Importing the libraries

In [ ]:
import time
import geopandas as gp 
import re
import numpy as np
import osmnx as ox
from scipy import stats
import os
start_time = time.time()

## Download the road network and clean the edge data

#step 1 download road network

In [ ]:
# Change 'path' variable to the path where all output shape files will be stored.
path = input("Enter directory where all output files are stored.")

## Specifing the paths

In [ ]:
name='area'
netwotkpath=r'' # the network shapefile path
netwotkpath_g=netwotkpath + r'_for_g' # the network shapefile path
edgefolder=r'' # the initial edge shapefile path
nodespath=netwotkpath+r'nodes\nodes.shp'# the nodes shapefile path, we don't need to process the nodes file
edgepath=path + r'\edges.shp' # the new edge shapefle path

### Dummmy global variables for error checking

In [ ]:
global error_seeker, item_number
error_seeker = 0
item_number = 0

## Set the defalt max speed value for the different highway type

smot = speed of motorway (mot)
spri = speed of primary road (pri)
ssec = speed of secondary road (sec)
ster = speed of tertiary road (ter)
sres = speed of residential road (res)
sroa = speed of road (roa)
sunc = speed of unclassified (unc)
stru = speed of trunck road (tru)

reference (https://wiki.openstreetmap.org/wiki/Key:highway)

In [ ]:
smot= 50; spri= 50; ssec= 50; ster= 40; sres= 40; sroa= 30; sunc= 50; stru= 50

## Set the default value of capacity of highway

In [ ]:
mot= 2000; pri= 900; sec= 700; ter= 600; res= 400; roa= 700; unc= 900; tru= 1200

## Filter if the user wants the residential edges within the network

In [ ]:
custom_filter_with_res='["highway"~"motorway|motorway_link|trunk|trunk_link|primary|primary_link|secondary|secondary_link|tertiary|tertiary_link|road|unclassified|residential"]'
custom_filter_no_res='["highway"~"motorway|motorway_link|trunk|trunk_link|primary|primary_link|secondary|secondary_link|road"]'


In [ ]:
#For removing certain road type
def remove_script():
    remove_links = input("Remove residential, tertiary, tertiary links, unclassified and other minor road types? (y/n)")
    return remove_links

#In case of multiple classification choose highest type label
def single_highway_class_choice():
    remove_labels = input("Choose only highest highway type label for all edges? (y/n)")
    return remove_labels


In [ ]:
dum = False
while not dum:
    remove_res = remove_script()
    if remove_res == 'y' or remove_res == 'n':
        dum = True
    else:
        print("Invalid input")

dum = False
while not dum:
    remove_label = single_highway_class_choice()
    if remove_label == 'y' or remove_label == 'n':
        dum = True
    else:
        print("Invalid input")

## Input Cells to enter the geo coordinates

In [ ]:
# kwarg(north lat, south lat, east long, west long). Latitudes must be between -90 and 90.
# Longitudes must be between -180 and 180, east larger than west
north_lat = input("Enter north latitude boundary for network (to 4 decimal places)")
south_lat = input("Enter south latitude boundary for network (to 4 decimal places)")
west_long = input("Enter west longitude boundary for network (to 4 decimal places)")
east_long = input("Enter east longitude boundary for network (to 4 decimal places)")


### Filters (Please refer to the functions above)

In [ ]:
if remove_res == 'y':
    G=ox.core.graph_from_bbox(north_lat, south_lat, east_long, west_long ,network_type='drive',  # the polygon points
                           simplify=True, infrastructure='way["highway"]', custom_filter=custom_filter_no_res)
else:
    G = ox.core.graph_from_bbox(north_lat, south_lat, east_long, west_long, network_type='drive',  # the polygon points
                                simplify=True, infrastructure='way["highway"]', custom_filter=custom_filter_with_res)


reference: https://en.wikipedia.org/wiki/World_Geodetic_System#:~:text=The%20WGS%2084%20meridian%20of%20zero%20longitude%20is,and%20flattening%20f%20=%201/%20298.257%20223%20563.

In [ ]:
G_projected=ox.project_graph(G, to_crs={'init': 'epsg:4326'}) #set the projection coordinate system as WGS 1984(lat lon)
ox.save_graph_shapefile(G_projected, filename=netwotkpath) #get the shape file #[outpt 1]

print('step 1: shape file retrieval')
print("--- %s seconds ---" % (time.time() - start_time)) #No need for this for you

In [ ]:
edges=gp.GeoDataFrame.from_file(netwotkpath+'data/edges/edges.shp')
nodes=gp.GeoDataFrame.from_file(netwotkpath+'data/nodes/nodes.shp')
edges=edges[['from','highway','lanes','length','maxspeed','name','oneway','osmid','to','geometry']]
# add the capacity column 
edges['capacity']=None

edges.dropna(subset=['highway']) # delete none value
edges['lanes']=edges['lanes'].fillna('2') # set 1 to none value for lane number


In [ ]:
# Looks into ambigious datapoints and returns single value
def lane_num(df): # to ture the ['1','2'] to 1
    if len(df['lanes'])>2:
        try:
            return re.findall(r"\d+\.?\d*", df['lanes'])[0]
        except IndexError:
            print(df['lanes'])
            return '2'
    elif df['lanes']=='#VALUE!':
        return '2'
    else:
         return df['lanes']

In [ ]:
edges['lanes']=edges.apply(lambda x: lane_num(x),axis=1)


edges['lanes']=edges['lanes'].astype(np.float32)

Cleans up speed so that it's a single number, or  or turn the ['100','90'] to 100, or leave it as None, to be
Filled in in a later function highway_speed_using_mode


In [ ]:
def highway_speed_cleanup(df):
    global error_seeker, item_number
    item_number += 1

    if df['maxspeed'] is None:
        pass
    elif len(df['maxspeed']) >= 2:
        try:
            return re.findall(r"\d+\.?\d*", df['maxspeed'])[0]
        except IndexError:
            print('Indexing Error for road speed parsing encountered on item number {}. maxspeed field for item '
                  'contains: '.format(item_number))
            print(df['maxspeed'])
            return None
    elif df['maxspeed']=='#VALUE!':
        return None
    else:
        return df['maxspeed']
edges['maxspeed']=edges.apply(lambda x: highway_speed_cleanup(x),axis=1)


## Set the capacity default value 

In [ ]:
def highway_capacity(df): # to set default speed or turn the ['100','90'] to 100
    if 'motor' in df['highway']:
        return mot
    elif 'primary' in df['highway']:
        return pri
    elif 'secondary' in df['highway']:
        return sec
    elif 'tertiary' in df['highway']:
        return ter
    elif 'road' in df['highway']:
        return sroa
    elif 'unclassified' in df['highway']:
        return unc
    elif 'trunk' in df['highway']:
        return tru
    else:
         return res

In [ ]:
edges['capacity']=edges.apply(lambda x: highway_capacity(x),axis=1)

In [ ]:
def choose_highest_rank_highway_type(df):
    if isinstance(df['highway'], list):
        if 'motor' in df['highway']:
            return 'motorway'
        elif 'trunk' in df['highway']:
            return 'trunk'
        elif 'primary' in df['highway']:
            return 'primary'
        elif 'secondary' in df['highway']:
            return 'secondary'
        elif 'tertiary' in df['highway']:
            return 'tertiary'
        elif 'residential' in df['highway']:
            return 'residential'
        elif 'unclassified' in df['highway']:
            return 'unclassified'
        return 'road'
    else:
        return df['highway']


In [ ]:
if remove_label == 'y':
    edges['highway']=edges.apply(lambda x: choose_highest_rank_highway_type(x),axis=1

A function to estimate the speed limits for the roads missing the data, by deploying the modal values of similar raoda type.
In cases all the roads are missing values we use defalt speed limits

In [ ]:
def calculate_highway_speed_using_mode(df):
    highway_speed_dict = {}

    for i in range(len(df)):
        if df['maxspeed'][i] != None:
            if df['highway'][i] not in highway_speed_dict:
                highway_speed_dict[df['highway'][i]] = [df['maxspeed'][i]]
            else:
                highway_speed_dict[df['highway'][i]].append([df['maxspeed'][i]])

    for key in highway_speed_dict:
        for i in range(len(highway_speed_dict[key])):
            if isinstance(highway_speed_dict[key][i], list) is True:
                try:
                    highway_speed_dict[key][i] = int(highway_speed_dict[key][i][0])
                except TypeError:
                    pass
            else:
                try:
                    highway_speed_dict[key][i] = int(highway_speed_dict[key][i])
                except TypeError:
                    pass

        highway_speed_dict[key] = list(filter(lambda a: a != [None], highway_speed_dict[key]))
        highway_speed_dict[key] = int(stats.mode(highway_speed_dict[key])[0])

    no_modal_speed_road_type = []
    for i in range(len(df)):
        if df['maxspeed'][i] is None:
            try:
                df.at[i, 'maxspeed'] = highway_speed_dict[df['highway'][i]]
            except KeyError:
                no_modal_speed_road_type.append(df['highway'][i])
                df.at[i, 'maxspeed'] = None

    return df

In [ ]:
edges = calculate_highway_speed_using_mode(edges)

In [ ]:
edges.to_csv(edges.csv) # specify output for edges.csv directory
nodes.to_csv(nodes.csv) # specify the output directory

In [ ]:
edges.crs = {'init':'epsg:4326'} # set the projection system code
edges.to_file(edgepath) # save the shapefile of new edges
print('step 2: clean the edges data')
print("--- %s seconds ---" % (time.time() - start_time))